In [2]:
import json
import pandas as pd
#import requests as re
import matplotlib.pyplot as plt


In [3]:
trafficVolume = pd.read_csv('https://opendata.arcgis.com/datasets/2b40c7c4367848218ce037a2a04b3c95_0.csv?geometry=%7B%22xmin%22%3A-80.783%2C%22ymin%22%3A38.1%2C%22xmax%22%3A-73.752%2C%22ymax%22%3A39.597%2C%22type%22%3A%22extent%22%2C%22spatialReference%22%3A%7B%22wkid%22%3A4326%7D%7D&outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D')

In [4]:
trafficVolume.head()

,X,Y,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,...,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO
0,-8.533742e+06,4.784321e+06,1,S2009030042,3,Baltimore,0,None,Timonium Rd,CO,...,"3,952","3,923","4,064",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-8.514937e+06,4.719226e+06,2,B0740,2,Anne Arundel,0,None,King George St,MD,...,"10,842","11,090","10,981",37.0,"9,258",31,881,146,8,154
2,-8.570495e+06,4.653970e+06,3,B1761,8,Charles,0,None,Port Tobacco Rd,MD,...,"10,811","10,712","10,613",24.0,"6,915",112,"2,270",577,115,692
3,-8.542958e+06,4.766530e+06,4,S1999030111,3,Baltimore,0,None,Ramp 3 Fr Md 122 Eb To Ramp 2 (to Is695),RP,...,"5,201","6,540","6,581",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-8.520413e+06,4.779664e+06,5,S2012030129,3,Baltimore,0,None,Waltham Woods Rd,CO,...,"7,495","6,370","6,611",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
trafficVolume.groupby('COUNTY_DESC').size()

COUNTY_DESC
Allegany             30
Anne Arundel        918
Baltimore          1241
Baltimore City      765
Calvert             104
Caroline            108
Carroll             159
Cecil                51
Charles             123
Dorchester           94
Frederick           399
Garrett              47
Harford             301
Howard              461
Kent                 72
Montgomery         1037
Prince George's    1079
Queen Anne's        138
Somerset             44
St. Mary's           94
Talbot               78
Washington           67
Wicomico            212
Worcester           121
dtype: int64

In [6]:
annualTraffic = trafficVolume[['COUNTY_DESC', 'AADT_2015', 'AADT_2016', 'AADT_2017', 'AADT_2018']]

In [7]:
counties = ['Montgomery', "Prince George's", 'Frederick']
annualTraffic.COUNTY_DESC.isin(counties)
myCounties = annualTraffic[annualTraffic.COUNTY_DESC.isin(counties)]

In [8]:
myCounties.head()

,COUNTY_DESC,AADT_2015,AADT_2016,AADT_2017,AADT_2018
15,Frederick,221.0,232.0,243.0,244.0
44,Frederick,4230.0,4321.0,4432.0,4413.0
65,Frederick,3342.0,3413.0,3504.0,3475.0
68,Frederick,4981.0,5082.0,4930.0,4891.0
89,Frederick,NaN,NaN,3010.0,2981.0


In [9]:
myCounties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2515 entries, 15 to 7741
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   COUNTY_DESC  2515 non-null   object 
 1   AADT_2015    2441 non-null   float64
 2   AADT_2016    2441 non-null   float64
 3   AADT_2017    2509 non-null   float64
 4   AADT_2018    2514 non-null   float64
dtypes: float64(4), object(1)
memory usage: 117.9+ KB


In [10]:
myCounties.groupby('COUNTY_DESC').sum()

,AADT_2015,AADT_2016,AADT_2017,AADT_2018
COUNTY_DESC,,,,
Frederick,3915773.0,3970413.0,4187239.0,4164736.0
Montgomery,16501207.0,16693983.0,17385546.0,17247153.0
Prince George's,18735586.0,19476264.0,20020861.0,20132363.0


In [11]:
# sum may not be an accurate count, trying max instead

myCounties.groupby('COUNTY_DESC').max()

,AADT_2015,AADT_2016,AADT_2017,AADT_2018
COUNTY_DESC,,,,
Frederick,119182.0,116330.0,119121.0,118052.0
Montgomery,255600.0,260971.0,267232.0,265633.0
Prince George's,252835.0,247970.0,253921.0,252402.0


I'm switching from sum to max, because the traffic is measured by the number of cars passing a point on a road.  Sum adds up the number of cars at all points; thus, the same vehicle may be counted multiple times on the same trip.  Max would give us instead the point with the highest average annual traffic volume, a more realistic indicator of overall traffic volume.

In [12]:
trafficVols = myCounties.groupby('COUNTY_DESC').max()

In [13]:
trafficVols


,AADT_2015,AADT_2016,AADT_2017,AADT_2018
COUNTY_DESC,,,,
Frederick,119182.0,116330.0,119121.0,118052.0
Montgomery,255600.0,260971.0,267232.0,265633.0
Prince George's,252835.0,247970.0,253921.0,252402.0


In [14]:
trafficVols.to_csv("trafficVols.csv")